<a href="https://colab.research.google.com/github/slowandfast/DiagRecommdSystem_01/blob/main/EMR_%ED%85%8D%EC%8A%A4%ED%8A%B8%EA%B8%B0%EB%B0%98_%EC%A7%84%EB%8B%A8%EC%B6%94%EC%B2%9C%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🧪 사전 작업

In [ ]:
!pip install pandas
!pip install openpyxl

In [ ]:
# emr dataset load

import pandas as pd

#file_path = "/content/[샘플]외래초진_신장내과_20240101_20240110_자료검토.xlsx"
file_path = "/content/신장내과_본원_외래_20230701_20231231.xlsx"

# 질병 코드 3개만 추림 (N029, N391, Z524)
# file_path = '/content/(샘플)외래초진_신장내과_20240101_20240110_자료검토_질병코드별분류.xlsx'

selected_columns = ['현병력'] # PI : 현병력
df = pd.read_excel(file_path, usecols=selected_columns)

df.head()


,현병력
0,115-61-61_x000D_\n2023.5.10._x000D_\n_x000D_\n...
1,"159/69/53_x000D_\n요증상(-), edema(-), 요독증상(-), D..."
2,"130/61/69_x000D_\n요증상(-), edema(-), 요독증상(-), D..."
3,"153/75/85_x000D_\n요증상(-), edema(-), 요독증상(-), D..."
4,"148/90/64_x000D_\n요증상(-), edema mild_x000D_\nC..."


In [ ]:
#!git clone https://github.com/rkiee/project-emrkcd-stopwords.git
!git clone https://github.com/rkiee/project-emr2kcd-dataset.git

Cloning into 'project-emr2kcd-dataset'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 3.52 MiB | 8.34 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
# stopwords dataset load
sw_path = "/content/project-emr2kcd-dataset/stopwords.xlsx"
sw_df = pd.read_excel(sw_path)
for i in range(len(sw_df)) :
  stop_words = sw_df['stop words'].tolist()
# print(stop_words)

In [ ]:
!pip install konlpy

In [ ]:
# 품사 빈도 조사
from konlpy.tag import Okt
okt = Okt()
all_parts = []
for n in range(len(df)) :
  text = df['PI'].iloc[n]
  words = okt.pos(text)
  parts_arr = []
  for i in words:
    if i == "EOS" or i == "" : continue
    word_temp = i[0]
    part = i[1]
    parts_arr.append(part)
  all_parts.extend(parts_arr)

all_parts_df = pd.DataFrame({"parts": all_parts, "count": len(all_parts) * [1]})
all_parts_df = all_parts_df.groupby("parts").sum()
all_parts_df = all_parts_df.sort_values("count", ascending=False)
print(all_parts_df)

                count
parts                
Alpha            9396
Punctuation      8159
Noun             5083
Number           4949
Foreign          2884
Josa              960
Verb              727
Adjective         360
Suffix            322
Modifier          200
Adverb             82
Determiner         73
Exclamation        27
VerbPrefix         17
Hashtag             2
URL                 1
Eomi                1
KoreanParticle      1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# stopwords 필터링 & 영단어(Alpha)와 명사(Noun)에 해당하는 형태소만 추출
from konlpy.tag import Okt
okt = Okt()

parts = ['Alpha', 'Noun']
all_words = []

for _, text in enumerate(df['PI']):
    words = okt.pos(text)  # 형태소, 품사 의 형태로 tokenization
    words_arr = []
    for i in words:
        if i == 'EOS' or i == '':
            continue
        word = i[0]  # 형태소
        part = i[1]  # 품사
        if not (part in parts):
            continue
        if word in stop_words:
            continue
        if part == 'Alpha':
            word = word.lower()
        words_arr.append(word)
    all_words.extend(words_arr)
print(all_words)

all_words_df = pd.DataFrame({'words': all_words, 'count': len(all_words) * [1]})
all_words_df = all_words_df.groupby('words').sum()
all_words_df = all_words_df.sort_values('count', ascending=False)
print(all_words_df)

# csv out : 단어 및 빈도수 기록
# all_words_df.to_csv("tokenized_words.csv", index=True, encoding="utf-8-sig")

['sudden', 'hearing', 'loss', 'nos', 'unilateral', 'rbc', 'uf', 'crystal', 'general', 'condition', 'good', 'ecog', 'ps', 'htn', 'dm', 'lmc', 'lab', '신장', '레아', '티닌', '수치', '이상', '거품', '뇨', '혈뇨', '부종', 'body', 'weight', 'change', 'uti', 'symptom', 'medication', '순환기', '내과', '류마티스', '내과', '한약', '식품', '인물', '즙', '신부전', '가족', 'ent', 'hearing', 'loss', 'steroid', '청력', '손실', 'ent', '중순', '소변', 'menses', 'period', '옆구리', '통증', 'kidney', 'cyst', 'complicated', 'wbc', 'rbc', '단백뇨', 'general', 'condition', 'good', 'ecog', 'ps', 'htn', 'dm', 'lmc', 'lab', '신장', '레아', '티닌', '수치', '거품', '뇨', '혈뇨', '부종', 'body', 'weight', 'change', 'uti', 'symptom', 'medication', '한약', '식품', '인물', '즙', '생수', '안과', '백내장', '녹내장', 'htn', 'ankylosing', 'spondylitis', 'crohn', 's', 'ds', 'maxnophen', 'nausea', 'tylenol', 'edema', 'fatty', 'liver', '저하', '단백뇨', '진통', '소염제', '복용', 'cr', 'egfr', 'ob', 'edema', '요독증', 'doe', 'known', 'gout', 'lmc', 'cr', 'egfr', 'ml', 'min', 'htn', '혈압', 'tia', 'cr', 'egfr', 'cr', 'egfr', '



---



🧪 TF-IDF

In [ ]:
import pandas as pd

file_path = '/content/[샘플]외래초진_신장내과_20240101_20240110_자료검토.xlsx'
selected_columns = ['DIAGCD', 'TERMHNGNM', 'PI']
data = pd.read_excel(file_path, usecols=selected_columns)
data.head()

,DIAGCD,TERMHNGNM,PI
0,R318.001.00,기타 및 상세불명의 혈뇨,"상기 환자는 진단명[Sudden hearing loss NOS, Unilateral..."
1,R829.005.00,요 분석의 이상,"상기 환자는 진단명[Kidney, cyst, complicated](으)로 현재 치..."
2,N182.001.00,만성 신장질환(2기),# HTN (2019.)_x000D_\n# Ankylosing spondylitis...
3,N183.001.00,만성 신장질환(3기),"140/93/77_x000D_\n요증상(-), edema(-), 요독증상(-), D..."
4,N183.001.00,만성 신장질환(3기),# HTN (약 5년 전부터 혈압 높은 경향)_x000D_\n# TIA_x000D_...


In [ ]:
# 컬럼 확인
data.columns

Index(['DIAGCD', 'TERMHNGNM', 'PI'], dtype='object')

In [ ]:
# 전처리
# PI의 결측치가 있는 항목은 모두 제거
data = data[data['PI'].notnull()].reset_index(drop=True)
data.shape

(297, 3)

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Okt
# konlpy의 Okt 클래스를 사용하여 토큰화하는 함수 정의
def okt_tokenizer(text):
  okt = Okt()
  tokens = okt.pos(text)  # 품사 태깅 수행
  # 명사(Noun) 및 영어 알파벳(Alpha)만 추출하여 리스트에 저장
  selected_tokens = [word for word, pos in tokens if pos in ['Noun', 'Alpha']]
  return selected_tokens

In [ ]:
sw_path = '/content/project-emr2kcd-dataset/stopwords.xlsx'
sw_df = pd.read_excel(sw_path)
for i in range(len(sw_df)) :
  stop_words = sw_df['stop words'].tolist()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, stop_words=stop_words, max_features=1000)

# PI에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['PI'])
tfidf_matrix.shape

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['a', 'b', 'g', 'm', 'p', 'r', '곽', '박경', '성은', '실', '입원', '효'] not in stop_words.
  warnings.warn(


(297, 1000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 만일 여기서 메모리 에러가 발생한다면, TF-IDF의 파라미터를 수정
# tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# 그래도, 안되는 경우에는 문서의 수를 조금 줄여서 실행
# data = data.loc[0:10000].reset_index(drop=True)

In [ ]:
import numpy as np
np.round(cosine_matrix, 4)

array([[1.    , 0.5862, 0.0098, ..., 0.    , 0.0116, 0.0268],
       [0.5862, 1.    , 0.0461, ..., 0.    , 0.0248, 0.0258],
       [0.0098, 0.0461, 1.    , ..., 0.0295, 0.0126, 0.0131],
       ...,
       [0.    , 0.    , 0.0295, ..., 1.    , 0.0492, 0.055 ],
       [0.0116, 0.0248, 0.0126, ..., 0.0492, 1.    , 0.5727],
       [0.0268, 0.0258, 0.0131, ..., 0.055 , 0.5727, 1.    ]])

In [ ]:
# id와 DIAGCD를 매핑할 dictionary를 생성
id2diagcd = {}
for i, c in enumerate(data['DIAGCD']):
  id2diagcd[i] = c

len(id2diagcd)

297

In [ ]:
# id2diagcd
for i, (id, diagcd) in enumerate(id2diagcd.items()):
  print(f"id: {id}, diagcd: {diagcd}")
  # Break the loop after printing the first 11 items (0 to 10)
  if i == 9:
    break

id: 0, diagcd: R318.001.00
id: 1, diagcd: R829.005.00
id: 2, diagcd: N182.001.00
id: 3, diagcd: N183.001.00
id: 4, diagcd: N183.001.00
id: 5, diagcd: N183.001.00
id: 6, diagcd: Z524.000.01
id: 7, diagcd: N391.000.01
id: 8, diagcd: N391.000.01
id: 9, diagcd: N049.000.00


In [ ]:
# DIAGCD와 id를 매핑할 dictionary를 생성
diagcd2id= {}
for i, c in id2diagcd.items(): diagcd2id[c] = i
len(diagcd2id)

47

In [ ]:
# diagcd2id
for i, (diagcd, id) in enumerate(diagcd2id.items()):
  print(f"diagcd: {diagcd}, id: {id}")
  # Break the loop after printing the first 10 items
  if i == 9:
      break

diagcd: R318.001.00, id: 0
diagcd: R829.005.00, id: 147
diagcd: N182.001.00, id: 259
diagcd: N183.001.00, id: 292
diagcd: Z524.000.01, id: 294
diagcd: N391.000.01, id: 290
diagcd: N049.000.00, id: 230
diagcd: N185.007.00, id: 296
diagcd: N178.011.00, id: 256
diagcd: N029.000.01, id: 284


In [ ]:
# id와 PI를 매핑할 dictionary
id2pi = {}
for i, c in enumerate(data['PI']): id2pi[i] = c

In [ ]:
def kcd_recommned(input) :
  okt = Okt()
  tokens = okt.pos(input)  # 품사 태깅 수행
  # 명사(Noun) 및 영어 알파벳(Alpha)만 추출하여 리스트에 저장
  input_data = [word for word, pos in tokens if pos in ['Noun', 'Alpha'] and word not in stop_words]

  query_sm = tfidf.transform(input_data)
  query_sm_arr = query_sm.toarray()

  sim = cosine_similarity(query_sm_arr, tfidf_matrix.toarray())
  sim = sim.reshape(297)

  sim_idx = sim.argsort()[::-1]

  for idx in sim_idx[:5] :
    print(f'{id2diagcd[idx]} ({sim[idx]}) id:{idx}')

kcd_recommned(input("search query input : "))



---



🧪 Doc2Vec

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install konlpy

In [ ]:
# stopwords 및 library에 추가할 단어 목록이 저장된 git storage 복사
!git clone https://github.com/rkiee/project-emr2kcd-dataset.git

In [ ]:
# okt library에 단어 추가
# 패키지에 설치된 java 경로에 jar파일 압축 풀기
import os

# print(os.getcwd())
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
# extract file
!jar xvf open-korean-text-2.1.0.jar

In [ ]:
# nouns update
# okt lib의 noun/nouns.txt의 단어 목록 로드
nouns_path = '/usr/local/lib/python3.10/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/'
nouns_df = []
nouns_df = pd.read_csv(os.path.join(nouns_path, "nouns.txt"), names = ["words"], header = None)

# 의약품 단어 목록 추가
words_path = '/content/project-emr2kcd-dataset/dictionary_medication.xlsx'
selected_columns = ['words']
w_df = []
w_df = pd.read_excel(words_path, usecols=selected_columns)
nouns_df = pd.concat([nouns_df, w_df], ignore_index=True)

# 일반 단어 목록 추가
words_path = '/content/project-emr2kcd-dataset/dictionary.xlsx'
selected_columns = ['words']
w_df = []
w_df = pd.read_excel(words_path, usecols=selected_columns)
nouns_df = pd.concat([nouns_df, w_df], ignore_index=True)

# 단어 목록 정렬
nouns_df = nouns_df.sort_values("words", ascending=True)

# txt 파일 형태로 저장
dictionary = ''
for i, text in enumerate(nouns_df['words']):
    dictionary += text + '\n'

with open("/usr/local/lib/python3.10/dist-packages/konlpy/java/org/openkoreantext/processor/util/noun/nouns.txt", 'w', encoding='utf-8') as f:
    f.write(dictionary)

In [ ]:
# 파일 묶기 : cvf
!jar cvf open-korean-text-2.1.0.jar org
# os.getcwd()
# os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
# remove
!rm -r org

In [ ]:
# emr dataset 로드
file_path = '/content/신장내과_본원_외래_20230701_20231231.xlsx'
selected_columns = ['ICD10', 'PI']
df = pd.read_excel(file_path, usecols=selected_columns)

In [ ]:
# stopwords 로드
stop_words = []
sw_path = '/content/project-emr2kcd-dataset/stopwords.xlsx'
sw_df = pd.read_excel(sw_path)
for i in range(len(sw_df)):
    stop_words = sw_df['stop words'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split

# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_test_val = train_test_split(df, test_size=0.3, random_state=42)
X_test, X_val = train_test_split(X_test_val, test_size=0.3, random_state=42)

# X_train : 학습 데이터 약 679 개 (이전 : 206개)
# X_test : 평가 데이터 약 292 개
# X_val : 검증(validatioin) 데이터 약 88 개

# print(f"학습용 데이터 갯수: {len(X_train)}")
# print(f"평가용 데이터 갯수: {len(X_test_val)} (X_test : {len(X_test)} + X_val : {len(X_val)})")

In [ ]:
# print(X_train.PI)

In [ ]:
# print(X_test_val.PI)

In [ ]:
from konlpy.tag import Okt
okt = Okt()

# Doc2Vec 모델 학습을 위해 데이터 포맷 변환 (TaggedDocument 객체 생성)
parts = ['Alpha', 'Noun']
tagged_data = []

for i, text in enumerate(df['PI']):
    morphs = okt.pos(text)  # 형태소 분석 결과를 가져옴
    filtered_word = []
    for morph, part in morphs:
        if part in parts and morph not in stop_words:
            filtered_word.append(morph)  # 선택한 품사에 해당하면서 불용어가 아닌 단어를 추가
    tagged_data.append(TaggedDocument(words=filtered_word, tags=[df.loc[i, 'ICD10']]))

# 현병력 형태소 분석 결과 출력
# print(tagged_data)

In [ ]:
# Doc2Vec 모델 생성 및 학습
model = Doc2Vec(vector_size=20, window = 1, min_count=1, epochs=50)
# 모델에 문서 집합을 토큰화하여 단어장을 생성
model.build_vocab(tagged_data)
# 학습
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
import numpy as np

# 각 제목 별 문서 벡터의 평균을 계산
title_vectors = {}
# 전체 목록을 돌면서
# ICD10 값 기준으로 나눔
for index, row in df.iterrows():
    vector = model.infer_vector(row['PI'].split())
    if row['ICD10'] not in title_vectors:
        title_vectors[row['ICD10']] = [vector]
    else:
        title_vectors[row['ICD10']].append(vector)
# print(title_vectors)

averaged_documents = []
for title, vectors in title_vectors.items():
    averaged_vector = np.mean(vectors, axis=0)
    averaged_documents.append((title, averaged_vector))
# print(averaged_documents)

# print(len(averaged_documents))

# 평균화된 벡터를 사용하여 모델 재학습
for title, vector in averaged_documents:
    # print(title)
    model.dv[title] = vector

# 모델 저장
model.save("/content/d2v.model")

In [ ]:
# 입력된 내용과 유사한 문서 검색
input_text = input("현병력 입력하세요: ")

morphs = okt.pos(input_text)
input_tokens = []
for morph, pos in morphs:
    if pos in parts and morph not in stop_words:
        input_tokens.append(morph)

print(input_tokens)
input_vector = model.infer_vector(input_tokens)

similar_documents = model.dv.most_similar([input_vector], topn=5)

for title, score in similar_documents:
    print(f"{title} similarity: {score:.3f}")

# for i in range(len(similar_documents)) :
#     print(similar_documents[i])

현병력 입력하세요: 상기 환자는 진단명[DJD](으)로 현재 치료중인 84세(F)의 환자입니다. 상기 병력 있는 환자로 Rt knee DJD 진단하 Rt TKRA 진행 계획중에 있습니다. 귀과적 병력 없는 분으로, 금번 시행한 pre op lab 상 BUN/Cr/eGFR 38.5/1.84/26 확인되어, 상기 소견 하 본과적 수술시행 전 Perioperative evaluation 및 수술 전후 환자 care & management 위한 귀과적 고견을 여쭙고자 의뢰드리오니 부디 고진선처 부탁드립니다. 감사합니다. =================================== 30년 전부터 CKD 알고 있음. 1/8 수술 예정
['DJD', 'Rt', 'knee', 'DJD', 'Rt', 'TKRA', '진행', '계획', '시행', 'pre', 'op', 'lab', 'BUN', 'Cr', 'eGFR', '수술', '시행', 'Perioperative', 'evaluation', '수술', 'care', 'management', 'CKD', '수술', '예정']
N184 similarity: 0.973
N183 similarity: 0.950
E118 similarity: 0.945
Q964 similarity: 0.931
J358 similarity: 0.930


In [ ]:
# input_vector = model.infer_vector(input_tokens)

# similar_documents = model.dv.most_similar([input_vector], topn=5)

# for title, score in similar_documents:
#     print(f"{title} similarity: {score:.3f}")

N184 similarity: 0.971
E118 similarity: 0.951
N183 similarity: 0.949
J358 similarity: 0.934
J342 similarity: 0.932


In [ ]:
# X_train : 학습 데이터 206
# X_test : 평가 데이터 89
# X_val : 검증(validatioin) 데이터 27

# 검증용 데이터를 사용하여 정확도 측정
y_true = [] # 실제값
y_pred = [] # 예측값

for i, row in X_val.iterrows():
    text = row['PI']
    morphs = okt.pos(text)
    input_tokens = []
    for morph, part in morphs:
        if part in parts and morph not in stop_words:
            input_tokens.append(morph)
    # print(input_tokens)

    input_vector = model.infer_vector(input_tokens) # 모델에 입력되는 벡터를 추론

    # X_train의 각 문서에 대해 입력벡터와의 유사도를 계산
    similarities = []
    for j, row_train  in X_train.iterrows():
        # j번째 문서에 대한 docment vector
        document_vector = model.dv[row_train['ICD10']]
        # input_vector와 document_vector 사이의 유사도 계산
        similarity = cosine_similarity([input_vector], [document_vector])[0][0]
        # # 계산된 유사도를 리스트에 추가
        similarities.append(similarity)

    # 가장 높은 유사도를 가진 doc의 인덱스를 가져온다
    predicted_index = np.argmax(similarities)
    # print(predicted_index)
    # 실제값을 y_true 리스트에 추가하고, 예측값을  y_pred 리스트에 추가
    y_true.append(row['ICD10'])
    y_pred.append(X_train.iloc[predicted_index]['ICD10'])

print(f"예측값 : {y_pred}")
print(f"실제값 : {y_true}\n")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division='warn')
f1 = f1_score(y_true, y_pred, average='weighted', zero_division='warn')
recall = recall_score(y_true, y_pred, average='weighted', zero_division='warn')

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 Score: {f1:.3f}")

# 측정 결과 범위
# Accuracy: 0.593 ~ 0.667
# Precision: 0.556 ~ 0.667
# Recall: 0.593 ~ 0.667
# F1 Score: 0.568 ~ 0.654

# Accuracy: 0.705 ~ 0.761
# Precision: 0.864 ~ 0.912
# Recall: 0.705 ~ 0.761
# F1 Score: 0.765 ~ 0.810

예측값 : ['C640', 'N182', 'N391', 'R311', 'C679', 'R310', 'C640', 'R798', 'N185', 'N391', 'N028', 'N185', 'N178', 'N059', 'J3809', 'N183', 'N181', 'N391', 'N183', 'Z524', 'N185', 'M3529', 'N184', 'N059', 'N184', 'N182', 'N391', 'N185', 'N184', 'N178', 'N183', 'N183', 'N184', 'N178', 'N182', 'N178', 'R311', 'N178', 'N183', 'N185', 'N183', 'N183', 'N183', 'N185', 'N391', 'N391', 'N184', 'N183', 'N183', 'N184', 'E118', 'N178', 'N183', 'Q612', 'N185', 'N183', 'N185', 'N178', 'R808', 'C900', 'N281', 'Z940', 'N183', 'N183', 'N391', 'Z940', 'R829', 'N183', 'N042', 'Z524', 'N182', 'R311', 'N281', 'N183', 'Z940', 'R609', 'N185', 'N281', 'N391', 'N183', 'N178', 'N391', 'N281', 'N178', 'N185', 'C679', 'R609', 'N185']
실제값 : ['C641', 'N182', 'N391', 'R311', 'N183', 'R310', 'N183', 'R798', 'N185', 'N391', 'N028', 'N185', 'N178', 'N391', 'N183', 'N183', 'N059', 'N391', 'N183', 'Z524', 'N185', 'N281', 'N184', 'N183', 'N184', 'N182', 'N391', 'N185', 'N184', 'N178', 'N183', 'N183', 'N184', 'N178', 'N183', 

<성능 평가 방법>

*   정확도 (Accuracy) [sklearn.metrics.accuracy_score](https://scikit-learn.org/0.15/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)
*   정밀도 (Precision) [sklearn.metrics.precision_score](https://scikit-learn.org/0.15/modules/generated/sklearn.metrics.precision_score.html)
*   재현율 (Recall) [sklearn.metrics.recall_score](https://scikit-learn.org/0.15/modules/generated/sklearn.metrics.recall_score.html#sklearn.metrics.recall_score)
*   F1 score [sklearn.metrics.f1_score](https://scikit-learn.org/0.15/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score)

---

*   정밀도, 재현율 : 불균형 데이터(클래스별 데이터 분포가 고르지 않은 경우)에서의 성능 측정을 위해 사용되는 지표
*   정밀도 : 긍정으로 예측한 대상 중에서 실제로 긍정인 데이터의 비율. 예) 스팸 메일 판단(음성을 양성으로 판정하면 업무 메일을 못 받음)의 경우 정밀도가 중요
*   재현율 : 실제 긍정인 데이터 중에서 긍정으로 예측한 대상의 비율. 예) 암 진단(양성을 음성으로 판정하면 큰 문제)과 같은 경우 재현율이 중요
*   F1 score : 정밀도와 재현율을 결합한 지표. 정밀도와 재현율 중 어느 한쪽으로 치우치지 않을수록 높은 값







